In [1]:
import pyexasol
import pandas as pd
import boto3
import time
import io
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

dwh_host = 'ec2-18-198-93-169.eu-central-1.compute.amazonaws.com'
#dwh_host = 'ip-172-31-40-248.eu-central-1.compute.internal'
dwh_port = '8563'
dwh_user = 'sys'
dwh_pass = 'Kartoffelpuff3r'

#connect to db
dwh_conn = pyexasol.connect(dsn=dwh_host + ':' + dwh_port, user=dwh_user, password=dwh_pass)


In [2]:

#get iterator month data
def get_iterator_data():

    #get iterator data
    v_sql = """SELECT distinct
        to_char(his.MATCH_DATE,'yyyymm') iterator
    FROM 
        BETTING_DV.FOOTBALL_MATCH_HIS_B his
        JOIN BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STRENGTH_FEATURES_EMA15_ha f
            ON his.FOOTBALL_MATCH_HIS_LID = f.FOOTBALL_MATCH_HIS_LID
        JOIN RAW_DV.FOOTBALL_MATCH_HIS_L_S_ODDS o
            ON his.FOOTBALL_MATCH_HIS_LID = o.FOOTBALL_MATCH_HIS_LID
    WHERE
        --2014_15 season dropped for stable features
        season <> '2014_2015'
        --2015_16 is first training season
        AND season <> '2015_2016'
        and season <> '2021_2022'
    ORDER BY 1;"""

    return dwh_conn.export_to_pandas(v_sql)

def get_training_data_comb(month_filter):

    #load training data based on provided filter
    v_sql = """-- BETTING_MART.UNDERSTAT_TEAM_MODEL_FEATURES_EMA15 source
    SELECT	
        --model features
        f.HOME_GOALS_FOR_STRENGTH,
        f.HOME_GOALS_AGAINST_STRENGTH,
        f.HOME_XG_FOR_STRENGTH,
        f.HOME_XG_AGAINST_STRENGTH,
        f.HOME_SHOTS_FOR_STRENGTH,
        f.HOME_SHOTS_AGAINST_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.HOME_PPDA_FOR_STRENGTH,
        f.HOME_PPDA_AGAINST_STRENGTH,
        f.HOME_DEEP_FOR_STRENGTH,
        f.HOME_DEEP_AGAINST_STRENGTH,
        f.AWAY_GOALS_FOR_STRENGTH,
        f.AWAY_GOALS_AGAINST_STRENGTH,
        f.AWAY_XG_FOR_STRENGTH,
        f.AWAY_XG_AGAINST_STRENGTH,
        f.AWAY_SHOTS_FOR_STRENGTH,
        f.AWAY_SHOTS_AGAINST_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.AWAY_PPDA_FOR_STRENGTH,
        f.AWAY_PPDA_AGAINST_STRENGTH,
        f.AWAY_DEEP_FOR_STRENGTH,	
        f.AWAY_DEEP_AGAINST_STRENGTH,
        --prediction class
        case 
          when s.HOME_GOALS  > s.AWAY_GOALS then 'H'
          when s.HOME_GOALS = s.AWAY_GOALS then 'D'
          else 'A'
        end match_result
    FROM 
        BETTING_DV.FOOTBALL_MATCH_HIS_B his
        JOIN BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STRENGTH_FEATURES_EMA15 f
            ON his.FOOTBALL_MATCH_HIS_LID = f.FOOTBALL_MATCH_HIS_LID
        JOIN RAW_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STATS s
            ON his.FOOTBALL_MATCH_HIS_LID = s.FOOTBALL_MATCH_HIS_LID
    WHERE
        --features have to exist
        f.HOME_GOALS_FOR_STRENGTH IS NOT NULL AND
        f.HOME_SHOTS_FOR_STRENGTH IS NOT NULL and
        f.AWAY_GOALS_FOR_STRENGTH IS not NULL and
        f.AWAY_SHOTS_FOR_STRENGTH IS not NULL and
        --first season is skipped to have stable features
        his.SEASON <> '2014_2015' and
        --filter for iterator
        to_number(to_char(his.MATCH_DATE,'yyyymm')) < """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)


def get_training_data_ha(month_filter):

    #load training data based on provided filter
    v_sql = """-- BETTING_MART.UNDERSTAT_TEAM_MODEL_FEATURES_EMA15 source
    SELECT	
        --model features
        f.HOME_GOALS_FOR_STRENGTH,
        f.HOME_GOALS_AGAINST_STRENGTH,
        f.HOME_XG_FOR_STRENGTH,
        f.HOME_XG_AGAINST_STRENGTH,
        f.HOME_SHOTS_FOR_STRENGTH,
        f.HOME_SHOTS_AGAINST_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.HOME_PPDA_FOR_STRENGTH,
        f.HOME_PPDA_AGAINST_STRENGTH,
        f.HOME_DEEP_FOR_STRENGTH,
        f.HOME_DEEP_AGAINST_STRENGTH,
        f.AWAY_GOALS_FOR_STRENGTH,
        f.AWAY_GOALS_AGAINST_STRENGTH,
        f.AWAY_XG_FOR_STRENGTH,
        f.AWAY_XG_AGAINST_STRENGTH,
        f.AWAY_SHOTS_FOR_STRENGTH,
        f.AWAY_SHOTS_AGAINST_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.AWAY_PPDA_FOR_STRENGTH,
        f.AWAY_PPDA_AGAINST_STRENGTH,
        f.AWAY_DEEP_FOR_STRENGTH,	
        f.AWAY_DEEP_AGAINST_STRENGTH,
        --prediction class
        case 
          when s.HOME_GOALS  > s.AWAY_GOALS then 'H'
          when s.HOME_GOALS = s.AWAY_GOALS then 'D'
          else 'A'
        end match_result
    FROM 
        BETTING_DV.FOOTBALL_MATCH_HIS_B his
        JOIN BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STRENGTH_FEATURES_EMA15_ha f
            ON his.FOOTBALL_MATCH_HIS_LID = f.FOOTBALL_MATCH_HIS_LID
        JOIN RAW_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STATS s
            ON his.FOOTBALL_MATCH_HIS_LID = s.FOOTBALL_MATCH_HIS_LID
    WHERE
        --features have to exist
        f.HOME_GOALS_FOR_STRENGTH IS NOT NULL AND
        f.HOME_SHOTS_FOR_STRENGTH IS NOT NULL and
        f.AWAY_GOALS_FOR_STRENGTH IS not NULL and
        f.AWAY_SHOTS_FOR_STRENGTH IS not NULL and
        --first season is skipped to have stable features
        his.SEASON <> '2014_2015' and
        --filter for iterator
        to_number(to_char(his.MATCH_DATE,'yyyymm')) < """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)




def get_pred_data_comb(month_filter):

    #load training data based on provided filter
    v_sql = """-- BETTING_MART.UNDERSTAT_TEAM_MODEL_FEATURES_EMA15 source
    SELECT
        f.football_match_his_lid,
        --model features
        f.HOME_GOALS_FOR_STRENGTH,
        f.HOME_GOALS_AGAINST_STRENGTH,
        f.HOME_XG_FOR_STRENGTH,
        f.HOME_XG_AGAINST_STRENGTH,
        f.HOME_SHOTS_FOR_STRENGTH,
        f.HOME_SHOTS_AGAINST_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.HOME_PPDA_FOR_STRENGTH,
        f.HOME_PPDA_AGAINST_STRENGTH,
        f.HOME_DEEP_FOR_STRENGTH,
        f.HOME_DEEP_AGAINST_STRENGTH,
        f.AWAY_GOALS_FOR_STRENGTH,
        f.AWAY_GOALS_AGAINST_STRENGTH,
        f.AWAY_XG_FOR_STRENGTH,
        f.AWAY_XG_AGAINST_STRENGTH,
        f.AWAY_SHOTS_FOR_STRENGTH,
        f.AWAY_SHOTS_AGAINST_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.AWAY_PPDA_FOR_STRENGTH,
        f.AWAY_PPDA_AGAINST_STRENGTH,
        f.AWAY_DEEP_FOR_STRENGTH,	
        f.AWAY_DEEP_AGAINST_STRENGTH
    FROM 
        BETTING_DV.FOOTBALL_MATCH_HIS_B his
        JOIN BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STRENGTH_FEATURES_EMA15 f
            ON his.FOOTBALL_MATCH_HIS_LID = f.FOOTBALL_MATCH_HIS_LID
        JOIN RAW_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STATS s
            ON his.FOOTBALL_MATCH_HIS_LID = s.FOOTBALL_MATCH_HIS_LID
    WHERE
        --features have to exist
        f.HOME_GOALS_FOR_STRENGTH IS NOT NULL AND
        f.HOME_SHOTS_FOR_STRENGTH IS NOT NULL and
        f.AWAY_GOALS_FOR_STRENGTH IS not NULL and
        f.AWAY_SHOTS_FOR_STRENGTH IS not NULL and
        --first season is skipped to have stable features
        his.SEASON <> '2014_2015' and
        --filter for iterator
        to_number(to_char(his.MATCH_DATE,'yyyymm')) = """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)


def get_pred_data_ha(month_filter):

    #load training data based on provided filter
    v_sql = """-- BETTING_MART.UNDERSTAT_TEAM_MODEL_FEATURES_EMA15 source
    SELECT
        f.football_match_his_lid,
        --model features
        f.HOME_GOALS_FOR_STRENGTH,
        f.HOME_GOALS_AGAINST_STRENGTH,
        f.HOME_XG_FOR_STRENGTH,
        f.HOME_XG_AGAINST_STRENGTH,
        f.HOME_SHOTS_FOR_STRENGTH,
        f.HOME_SHOTS_AGAINST_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.HOME_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.HOME_PPDA_FOR_STRENGTH,
        f.HOME_PPDA_AGAINST_STRENGTH,
        f.HOME_DEEP_FOR_STRENGTH,
        f.HOME_DEEP_AGAINST_STRENGTH,
        f.AWAY_GOALS_FOR_STRENGTH,
        f.AWAY_GOALS_AGAINST_STRENGTH,
        f.AWAY_XG_FOR_STRENGTH,
        f.AWAY_XG_AGAINST_STRENGTH,
        f.AWAY_SHOTS_FOR_STRENGTH,
        f.AWAY_SHOTS_AGAINST_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_FOR_STRENGTH,
        f.AWAY_SHOTS_ON_TARGET_AGAINST_STRENGTH,
        f.AWAY_PPDA_FOR_STRENGTH,
        f.AWAY_PPDA_AGAINST_STRENGTH,
        f.AWAY_DEEP_FOR_STRENGTH,	
        f.AWAY_DEEP_AGAINST_STRENGTH
    FROM 
        BETTING_DV.FOOTBALL_MATCH_HIS_B his
        JOIN BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STRENGTH_FEATURES_EMA15_ha f
            ON his.FOOTBALL_MATCH_HIS_LID = f.FOOTBALL_MATCH_HIS_LID
        JOIN RAW_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_STATS s
            ON his.FOOTBALL_MATCH_HIS_LID = s.FOOTBALL_MATCH_HIS_LID
    WHERE
        --features have to exist
        f.HOME_GOALS_FOR_STRENGTH IS NOT NULL AND
        f.HOME_SHOTS_FOR_STRENGTH IS NOT NULL and
        f.AWAY_GOALS_FOR_STRENGTH IS not NULL and
        f.AWAY_SHOTS_FOR_STRENGTH IS not NULL and
        --first season is skipped to have stable features
        his.SEASON <> '2014_2015' and
        --filter for iterator
        to_number(to_char(his.MATCH_DATE,'yyyymm')) = """ + str(month_filter) + ";"

    return dwh_conn.export_to_pandas(v_sql)


In [3]:

#truncate result table
dwh_conn.execute('truncate table BETTING_DV.FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15')

#get iterator data
df_iterator = get_iterator_data()

#
# combined models
#

#loop over month
for index, row in df_iterator.iterrows():
    print('Prediction for: ' + str(row['ITERATOR']))
    
    df_train_data = get_training_data_comb(row['ITERATOR'])
    
    #split features and pred classes
    df_train_x = df_train_data.iloc[:,:24]
    df_train_y = df_train_data.iloc[:,24:25]
    
    
    #
    #train logistic regression model
    #
    print('...training logistic regression model')
    
    solvers = 'lbfgs'
    penalty = 'l2'
    c_values = 0.01
       
    lr_model = LogisticRegression(solver=solvers, penalty=penalty, C=c_values, random_state=0).fit(df_train_x, df_train_y)
    
    #print('......training auc score: ' + str(roc_auc_score(df_train_y, xg_model.predict_proba(df_train_x),multi_class='ovo')))
    print('......training score: ' + str(lr_model.score(df_train_x,df_train_y)))
    
    #
    # train random forest model
    #
    
    print('...training random forest model')
    
    max_depth = 50
    min_samples_leaf = 10
    min_samples_split = 150
    max_leaf_nodes = 60
    n_estimators = 90
    max_features = 24
    
    # define models and parameters
    rf_model = RandomForestClassifier(max_depth = max_depth, min_samples_leaf=min_samples_leaf, 
                                      min_samples_split=min_samples_split, max_leaf_nodes=max_leaf_nodes,
                                      n_estimators=n_estimators,max_features=max_features
                                     ).fit(df_train_x, df_train_y)
    
    print('......training score: ' + str(rf_model.score(df_train_x,df_train_y)))
    
    #
    # train xgboost model
    #
    
    print('...training xgboost model')
    
    learning_rate = 0.07
    n_estimators = 105
    max_depth = 5
    min_child_weight = 12
    subsample = 0.9
    colsample_bytree = 0.8
    gamma  = 0.2
    
    xg_model = XGBClassifier(objective ='multi:softprob', learning_rate=learning_rate, n_estimators=n_estimators,
                            max_depth=max_depth, min_child_weight=min_child_weight,subsample=subsample,
                            colsample_bytree=colsample_bytree, gamma=gamma 
                            ).fit(df_train_x, df_train_y)
    
    print('......training score: ' + str(xg_model.score(df_train_x,df_train_y)))
    
    #
    # predicting logistic regression
    #
    
    #get data for prediction
    df_pred_data = get_pred_data_comb(row['ITERATOR'])
    
    #do prediction
    df_pred_x = df_pred_data.iloc[:,1:25]
    
    print('...executing logistic regression prediction')  
    
    df_pred_y = lr_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'Logistic Regression EMA15 combined'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    
    
    #
    # predicting random forest
    #
    
    print('...executing random forest prediction')  
    
    df_pred_y = rf_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'Random forest EMA15 combined'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    
    
    #
    # predicting xgboost
    #

    print('...executing xgboost prediction')  
    
    df_pred_y = xg_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'XGBoost EMA15 combined'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    

    
#
# HA models
#

#loop over month
for index, row in df_iterator.iterrows():
    print('Prediction for: ' + str(row['ITERATOR']))
    
    df_train_data = get_training_data_ha(row['ITERATOR'])
    
    #split features and pred classes
    df_train_x = df_train_data.iloc[:,:24]
    df_train_y = df_train_data.iloc[:,24:25]
    
    
    #
    #train logistic regression model
    #
    print('...training logistic regression model')
    
    solvers = 'liblinear'
    penalty = 'l2'
    c_values = 1.0
       
    lr_model = LogisticRegression(solver=solvers, penalty=penalty, C=c_values, random_state=0).fit(df_train_x, df_train_y)
    
    #print('......training auc score: ' + str(roc_auc_score(df_train_y, xg_model.predict_proba(df_train_x),multi_class='ovo')))
    print('......training score: ' + str(lr_model.score(df_train_x,df_train_y)))
    
    #
    # train random forest model
    #
    
    print('...training random forest model')
    
    max_depth = 20
    min_samples_leaf = 10
    min_samples_split = 150
    max_leaf_nodes = 60
    n_estimators = 50
    max_features = 24
    
    # define models and parameters
    rf_model = RandomForestClassifier(max_depth = max_depth, min_samples_leaf=min_samples_leaf, 
                                      min_samples_split=min_samples_split, max_leaf_nodes=max_leaf_nodes,
                                      n_estimators=n_estimators,max_features=max_features
                                     ).fit(df_train_x, df_train_y)
    
    print('......training score: ' + str(rf_model.score(df_train_x,df_train_y)))
    
    #
    # train xgboost model
    #
    
    print('...training xgboost model')
    
    learning_rate = 0.07
    n_estimators = 105
    max_depth = 5
    min_child_weight = 12
    subsample = 0.9
    colsample_bytree = 0.8
    gamma  = 0.2
    
    xg_model = XGBClassifier(objective ='multi:softprob', learning_rate=learning_rate, n_estimators=n_estimators,
                            max_depth=max_depth, min_child_weight=min_child_weight,subsample=subsample,
                            colsample_bytree=colsample_bytree, gamma=gamma 
                            ).fit(df_train_x, df_train_y)
    
    print('......training score: ' + str(xg_model.score(df_train_x,df_train_y)))
    
    #
    # predicting logistic regression
    #
    
    #get data for prediction
    df_pred_data = get_pred_data_ha(row['ITERATOR'])
    
    #do prediction
    df_pred_x = df_pred_data.iloc[:,1:25]
    
    print('...executing logistic regression prediction')  
    
    df_pred_y = lr_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'Logistic Regression EMA15 ha'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    
    
    #
    # predicting random forest
    #
    
    print('...executing random forest prediction')  
    
    df_pred_y = rf_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'Random forest EMA15 ha'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    
    
    #
    # predicting xgboost
    #

    print('...executing xgboost prediction')  
    
    df_pred_y = xg_model.predict_proba(df_pred_x)
    
    #create result data frame
    df_result_data = pd.DataFrame(df_pred_data.iloc[:,0:1])
    df_result_data['model'] = 'XGBoost EMA15 ha'
    df_result_data['home_win_prob'] = df_pred_y[:,2:3]
    df_result_data['draw_prob'] = df_pred_y[:,1:2]
    df_result_data['away_win_prob'] = df_pred_y[:,0:1]
    df_result_data['interator_param'] = str(row['ITERATOR'])
    
    print('......data written to the db')    
    dwh_conn.import_from_pandas(df_result_data,('BETTING_DV','FOOTBALL_MATCH_HIS_L_S_UNDERSTAT_TEAM_PRED_EMA15'))
    
    

Prediction for: 201608
...training logistic regression model
......training score: 0.5079977937120794
...training random forest model


C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\ipykernel_launcher.py:53: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


......training score: 0.5763927192498621
...training xgboost model
[23:19:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


......training score: 0.8268063982349697
...executing logistic regression prediction
......data written to the db
...executing random forest prediction
......data written to the db
...executing xgboost prediction
......data written to the db
Prediction for: 201609
...training logistic regression model
......training score: 0.5122843700993205
...training random forest model


...

......training score: 0.5278685373822795
...training random forest model


C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\ipykernel_launcher.py:191: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


......training score: 0.5652508168364405
...training xgboost model
[00:01:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\andre\.conda\envs\Betting DB\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


......training score: 0.6054199500288295
...executing logistic regression prediction
......data written to the db
...executing random forest prediction
......data written to the db
...executing xgboost prediction
......data written to the db


In [96]:

#clean up


#close db conncection
dwh_conn.close()
